# Driver Aggression Neural Network (DANN)

Driver Aggression Neural Network is assigning an aggression value to a sorted set of sensory data. Driving is simulated in BeamNG v0.27 using their BeamNGpy open-source library.

In [1]:
csv_file_path = 'imu_data_2023_04_29_17_22_10.csv'

In [2]:
import numpy as np
from matplotlib import pyplot as plt

import torch

In [3]:
import pandas as pd

data = pd.read_csv(csv_file_path)
print(data.head())

                                                                                                                                                                                                                                                                                                                                                                                                                                                      iterationId;imuId;vehicleId;time;pos;dirX;dirY;dirZ;angVel;angAccel;mass;accRaw;accSmooth
0;vehicle_0_main_imu;vehicle_0_main;21.37300101...  101.125750  120.00966406065];[-0.7063269517349199  -0.707881  -0.0025598254149101006];[0.005078296606543902 -0.001451  -0.9999860526255101];[0.7078675357343299 -0.706330  0.0046197245945999];[0.019709264337255002   0.001809  0.026435184728145007];[0.25780630178655006  0.367080   0.16575583925473003];4.467717715826;[-0.486391...  1.288496  2.0623662656039];[0.2807396623313   1.012241                                   